# Importación de librerías y datos

In [10]:

import pandas as pd
import sqlite3
from surprise import Dataset, Reader, dump
from surprise.model_selection import cross_validate, train_test_split, GridSearchCV
from surprise.prediction_algorithms import BaselineOnly, SVD, KNNBasic


from recomendar import DATABASE_FILE

conn = sqlite3.connect(DATABASE_FILE)
df = pd.read_sql("SELECT * FROM interacciones", conn)
conn.close()


print(df.head())
print(f"Número de filas: {len(df)}")


             id_usuario        id_juego  score
0  attack-of-the-fanboy  baldurs-gate-3    100
1            gamesradar  baldurs-gate-3    100
2           gamingtrend  baldurs-gate-3    100
3               ggrecon  baldurs-gate-3    100
4               gfinity  baldurs-gate-3    100
Número de filas: 345158


# Entrenar modelo SVD optimizando hiperparámetros con grid search

In [17]:
# %% [markdown]
# # Grid search de hiperparámetros para SVD (Surprise)
# - Búsqueda de mejores hiperparámetros con validación cruzada
# - Entrenamiento del modelo final con esos hiperparámetros
# - Guardado del modelo en SVD.pkl


# df ya viene cargado desde la celda anterior:
# columnas: id_usuario, id_juego, score

# Preparar dataset para Surprise
reader = Reader(rating_scale=(1, 100))
data = Dataset.load_from_df(df[["id_usuario", "id_juego", "score"]], reader)

# %%
# Definir el espacio de búsqueda de hiperparámetros
# Podés ajustar las listas si querés explorar más/menos combinaciones

param_grid = {
    "n_factors": [50, 100, 150, 500],   # dimensión del espacio latente
    "n_epochs": [10, 20, 30],      # número de épocas de entrenamiento
    "lr_all": [0.002, 0.005],      # learning rate global
    "reg_all": [0.02, 0.1],        # regularización global
}

# Grid search con validación cruzada (5-fold) optimizando RMSE
gs = GridSearchCV(
    algo_class=SVD,
    param_grid=param_grid,
    measures=["rmse"],
    cv=5,
    refit=True,     # entrena automáticamente el mejor modelo sobre todos los datos de CV
    n_jobs=-1       # usa todos los cores disponibles (si te da problemas, ponelo en 1)
)

# Esto entrena todos los modelos del grid
gs.fit(data)

# %%
# Mostrar mejores resultados
print("Mejores hiperparámetros encontrados:")
print(gs.best_params["rmse"])

print(f"Mejor RMSE en validación cruzada: {gs.best_score['rmse']:.4f}")

# El mejor modelo según RMSE:
best_algo = gs.best_estimator["rmse"]

# %%
# Entrenar modelo final sobre TODO el dataset (full trainset)
# (GridSearchCV con refit=True ya lo entrena sobre todo el 'trainset' de CV,
#  pero si querés asegurarte de usar TODAS las observaciones posibles,
#  podés rearmar un trainset completo manualmente.)

full_trainset = data.build_full_trainset()

final_model = SVD(
    n_factors=gs.best_params["rmse"]["n_factors"],
    n_epochs=gs.best_params["rmse"]["n_epochs"],
    lr_all=gs.best_params["rmse"]["lr_all"],
    reg_all=gs.best_params["rmse"]["reg_all"],
    random_state=42,
)

final_model.fit(full_trainset)

# %%
# Guardar el modelo entrenado en un archivo pickle
dump.dump('./SVD.pkl', algo=best_algo)

print(f"Modelo final guardado en: 'SVD.pkl'")


Mejores hiperparámetros encontrados:
{'n_factors': 500, 'n_epochs': 10, 'lr_all': 0.002, 'reg_all': 0.1}
Mejor RMSE en validación cruzada: 10.1417
Modelo final guardado en: 'SVD.pkl'


# Ejemplo de uso

In [15]:
id_usuario = 'ign' #8157 valorados
id_usuario = 'arcade-sushi' #218 valorados
id_juego = 'eco-shooter-plant-530' # jUEGO NO RELEVANTE PARA  da-gameboyz' 

pred = final_model.predict(uid=id_usuario, iid=id_juego)
print(pred.est)


69.24285002650728


# Exportar parámetros de SVD generado por Surprise para trabajarlo en Pythoneverywhere

In [21]:
import numpy as np
from surprise import dump

# === Cargar el modelo entrenado ===
_, algo = dump.load("./datos/SVD.pkl")

# === Mapeos de IDs internos ←→ IDs originales ===
uid_map = {algo.trainset.to_raw_uid(inner): inner for inner in algo.trainset.all_users()}
iid_map = {algo.trainset.to_raw_iid(inner): inner for inner in algo.trainset.all_items()}

# === Guardar todos los parámetros necesarios ===
np.savez(
    "svd_params.npz",
    pu=algo.pu,                 # matriz factores usuario (n_users × n_factors)
    qi=algo.qi,                 # matriz factores ítem    (n_items × n_factors)
    bu=algo.bu,                 # sesgo usuario
    bi=algo.bi,                 # sesgo ítem
    mu=algo.trainset.global_mean,  # media global
    uid_map=uid_map,
    iid_map=iid_map
)

print("Archivo guardado como svd_params.npz")


Archivo guardado como svd_params.npz
